## <span style="color:red">Accesso agli attributi e descrittori</span>

Un aspetto di Python che è necessario comprendere bene nei dettagli, per apprezzarne la ricchezza e le ampie possibilità di utilizzo, è l'insieme dei "maccanismi" di accesso agli attributi degli oggetti. In realtà, si tratta ben più del semplice "accesso"; sono mezzi che il linguaggio mette a disposizione per conferire _significato_ agli attributi.

Qualcosa abbiamo già visto, ma ora è necessario entrare in maggiori dettagli

#### Protocollo standard

In [1]:
class father:
    
    z = "A class attribute"

In [ ]:
class daughter(father):
    
    y = "Another class attribute"
    
    def __init__(self, x=''):
        self.x = x

Quando viene tentato l'accesso ad un attributo _a_ di un oggetto _x_, viene invocato il metodo <em>\_\_getattribute\_\_</em> di x (che, tipicamente, è definito nella classe <em>object</em>, classe base - diretta o indiretta - di qualsiasi altra classe in Python 3)

Il comportamento "osservabile" di \_\_getattribute\_\_ è il seguente

1. Se _a_ è una chiave presente nel dizionario x.\_\_dict\_\_ allora restituisce il corrispondente valore

In [ ]:
d = daughter('Ann')
print(d.x)
d.__dict__

2. Se _a_ è presente nel dizionario della classe o, in caso negativo, di una superclasse, allora restituisce il corrispondente valore

In [ ]:
print(d.y)
type(d).__dict__

In [ ]:
print(d.y)
father.__dict__

3. Se un attributo <u>non viene trovato</u> nel modo sopra esposto e nella classe <u>non è definito</u> il particolare metodo <em>\_\_getattr\_\_</em> allora solleva un'eccezione (errore di tipo <em>AttributeError</em>)

In [ ]:
d.w

4. Se un attributo <u>non viene trovato</u> ma nella classe <u>esiste</u> il metodo <em>\_\_getattr\_\_</em> allora invoca quest'ultimo (con l'attributo come parametro)

In [2]:
class daughter(father):
    
    y = "Another class attribute"
    
    def __init__(self, x=''):
        self.x = x
        
    def __getattr__(self,e):
        print(f"Attributo {e} non trovato")
        return None

In [4]:
d = daughter('Ann')
d.w

Attributo w non trovato


Non sorprende che, secondo la "filosofia Python", quanto esposto può essere modificato ridefinendo proprio il metodo <em>\_\_getattribute\_\_</em>

In [5]:
class esempio1(father):
    ''' Semplice classe che illustra il protocollo di accesso
        agli attributi mediante il metodo __getattribute__ esplicitamente
        riscritto
    '''
    
    y = "Another class attribute"
    
    def __init__(self, x=0):
        '''Ogni oggetto ha un solo attributo dinamico'''
        self.x = x
        
    def __getattribute__(self, attr):
        '''Esegue l'accesso all'attributo mediante il metodo in object'''
        print(f"Look up dell'attributo {attr}")
        return object.__getattribute__(self, attr)
    
    def __getattr__(self, attr):
        '''Gestisce l'eccezione "AttributeError" '''
        print(f"Se l'attributo {attr} non è definito viene chiamata __getattr__")
        return None

In [6]:
a = esempio1()
a.x           # Caso di attributo presente nell'oggetto

Look up dell'attributo x


0

In [8]:
a.y          # Caso di attributo presente nella classe

Look up dell'attributo y


'A class variable'

In [9]:
a.w           # Caso di attributo non definito

Look up dell'attributo w
Se l'attributo w non è definito viene chiamata __getattr__


#### Quando lo zucchero non è solo zucchero...

In [11]:
# Attenzione
a.__getattribute__('z') # Chiamata esplicita del metodo; risultato ok perché z esiste

Look up dell'attributo __getattribute__
Look up dell'attributo z


'A class attribute'

In [12]:
a.__getattribute__('w') # L'eccezione non viene intrappolata perché __getattribute__
                        # non chiama __getattr__

Look up dell'attributo __getattribute__
Look up dell'attributo w


AttributeError: 'esempio1' object has no attribute 'w'

a.\_\_getattribute\_\_('w') non è dunque completamente equivalente a scrivere a.w

Nel secondo caso è l'interprete che, quando \_\_getattribute\_\_ solleva l'eccezione, invoca il metodo di gestione \_\_getattr\_\_

Per avere completa equivalenza, con la riscrittura esplicita di \_\_getattribute\_\_, bisogna farsi carico di ciò

In [13]:
class esempio2(father):
    ''' Versione aderente ai punti 1-4 indicati sopra per il comportamento di
        __getattribute__
    '''
    
    y = "Another class attribute"
    
    def __init__(self, x = 0):
        '''Ogni oggetto ha un solo attributo dinamico'''
        self.x = x
        
    def __getattribute__(self, attr):
        ''' Implementa i punti 1-4 del protocollo definito sopra
        '''
        try:
            print(f"Look up dell'attributo {attr}")
            return object.__getattribute__(self,attr)
        except AttributeError:
            try:
                return object.__getattr__(self,attr)
            except AttributeError:
                print(f"L'attributo {attr} e il metodo __getattr__ non sono definiti")
                return None
    
    def __getattr__(self, attr):
        '''Gestisce l'eccezione "AttributeError" '''
        print(f"Se l'attributo {attr} non è definito viene chiamata __getattr__")
        return None

In [14]:
a = esempio2()
a.x

Look up dell'attributo x


0

In [15]:
a.__getattribute__('x')

Look up dell'attributo __getattribute__
Look up dell'attributo x


0

In [16]:
a.w

Look up dell'attributo w
L'attributo w e il metodo __getattr__ non sono definiti


In [17]:
a.__getattribute__('w')

Look up dell'attributo __getattribute__
Look up dell'attributo w
L'attributo w e il metodo __getattr__ non sono definiti


Il corrispondente, per la modifica (o la creazione) di un attributo, del metodo \_\_getattribute\_\_ non è \_\_setattribute\_\_ (che non esiste), bensì <em>\_\_setattr\_\_</em>

In [18]:
def __setattr__(self, name, value):
    self.__dict__[name] = value
    # Il semplice self.name = value provocherebbe ricorsione infinita

esempio2.__setattr__=__setattr__

In [19]:
a = esempio2()

Look up dell'attributo __class__
Look up dell'attributo __class__
Look up dell'attributo __class__
Look up dell'attributo __class__
Look up dell'attributo __class__
Look up dell'attributo __class__
Look up dell'attributo __dict__


In [20]:
a.w

Look up dell'attributo w
L'attributo w e il metodo __getattr__ non sono definiti


In [21]:
a.w = 5

Look up dell'attributo __dict__


In [22]:
a.w

Look up dell'attributo w


5

### Decoratore @property

Abbiamo visto che un modo "pythonico" di forzare alcuni vincoli sul valore di un certo attributo consiste nel definire tale attributo come _property_. Questo obiettivo può essere ottenuto usando il decoratore _@property_ che è associato a metodi getter e setter

Al riguardo, riprendiamo ancora una volta l'esempio della classe per la gestione di un conto corrente

In [ ]:
class CC:
    """
    Classe per la gestione di un c/c:: v2.01
    Semplicemente mette in evicenza scritture alternative
    per specificare che un dato attributo è in realtò una proprietà
    """
    def __init__(self, deposito_iniziale = 0):
        print("Apertura conto con {0:.2f} Euro".format(deposito_iniziale))
        self.saldo = deposito_iniziale
        
    def prelievo(self, importo=0):
        # Se importo==0 equivale ad una lettura del saldo
        # Per questa ragione prelievo (con importo == 0)
        # può essere usato come "getter"
        if importo>0 and self._saldo-importo>=0:
            self._saldo -= importo
        return self._saldo
    
    def __modifica_saldo(self, importo):
        # metodo usato come "setter"; forza il vincolo
        # sulla non negatività dell'importo
        if importo>=0:
            self._saldo = importo
      
    def versamento(self, importo):
        # versamento non può essere un "setter" perché
        # modifica il valore sulla base di quello attuale
        if importo>0:
            self._saldo += importo
     
    def mostra_saldo(self):
        print("Il saldo è di {0:.2f} Euro".format(self.__saldo))
    
    saldo=property(prelievo, __modifica_saldo) # getter e setter
    
    # Equivalente
    # saldo=property(fget=prelievo, fset=__modifica_saldo)
    # e dunque anche
    # saldo=property(fset=__modifica_saldo,fget=prelievo)
    
    # Altra versione
    # saldo = property()
    # saldo = saldo.getter(prelievo)
    # saldo = saldo.setter(__modifica_saldo)

In [ ]:
X = CC(50)

In [ ]:
X.saldo = 100
print(X.saldo)

In [ ]:
X.saldo = -10
print(X.saldo)

In [ ]:
class CC:
    """
    Classe per la gestione di un c/c:: v2.1
    Riorganizza i metodi allo scopo di utilizzare i decoratori
    """
    def __init__(self, deposito_iniziale = 0):
        print("Apertura conto con {0:.2f} Euro".format(deposito_iniziale))
        self._saldo = deposito_iniziale
    
    @property
    def saldo(self):
        return self._saldo
    
    @saldo.setter
    def saldo(self,value):
        if value>0:
            self._saldo = value
    
    def versamento(self, importo):
        if importo>0:
            self.saldo += importo
        
    def prelievo(self, importo=0):
        if importo>0 and self.saldo>=importo:
            self.saldo -= importo
    
    def mostra_saldo(self):
        print("Il saldo è di {0:.2f} Euro".format(self._saldo))

In [ ]:
X = CC(100)

In [ ]:
X.saldo = 100
X.mostra_saldo()

In [ ]:
X.saldo = -10
X.mostra_saldo()

In [ ]:
X.versamento(50)
X.mostra_saldo()

In [ ]:
X.prelievo(3)

### Descrittori

Un descrittore è una classe in cui è presente la definizione di almeno uno
dei seguenti _magic method_

1. __get__(self, instance, cls)
2. __set__(self, instance, value)
3. __delete__(self, instance).

I descrittori vengono utilizzati per controllare l'accesso agli attributi di altre classi. Sono quindi una generalizzazione delle proprietà

Vediamo subito un semplice esempio

In [ ]:
class descr:
    
    def __init__(self, name="attr", initval=None):
        self.name = name
        self.val = initval
        
    def __get__(self, inst, cls):
        '''
        In questo esempio, inst e cls non sono utilizzati.
        inst è l'oggetto in cui il descritore è utilizzato e cls 
        il suo tipo (classe). Il valore restituito potrebbe dipendere
        quindi da altri attributi definiti nell'oggetto in questione.
        '''
        print("__get__ called")
        return self.val
    
    def __set__(self, inst, val):
        '''
        Anche in questo caso l'oggetto (inst) non viene utilizzto
        '''
        print("__set__ called")
        if val < 0 or val > 1:
            raise ValueError(f"Value of {self.name} should be in the range [0,1]")
        else:
            self.val = val

In [ ]:
class test:
    v = descr('probability_value',0.0)

In [ ]:
t = test()
t.v = 0.6
print(t.v)

In [ ]:
t.v = -1

#### Qualcuno nota "qualcosa di strano"?

Iniziamo a fare qualche "sondaggio"

In [ ]:
t.__dict__

In [ ]:
type(t).__dict__

Effettivamente _v_ è un attributo di classe, ma allora...

In [ ]:
q = test()
print(q.v)

... c'è un solo valore possibile e dunque una sola "vera" istanza per la classe _test_?

L'esempio presentato è troppo semplice... Il fatto che un descrittore venga utilizzato come attributo di classe ha diversi vantaggi (ad esempio in termini di efficienza). E non pregiudica la possibilità di utilizzarlo per istanze diverse. Come vedremo subito nell'ultima versione della classe CC

In [ ]:
class balance:
    '''Classe il cui uso inteso è di memorizzare il (e operare sul)
       saldo di molteplici conti correnti. Poiché implementa i metodi
       __get__, __set__ e __delete__, la classe è un descrittore.
       La classe utilizza un dizionario (scelta migliorabile) per 
       memorizzare il saldo di conti correnti diversi. Le chiavi di accesso
       sono proprio gli oggetti (della classe CC) che rappresentano
       i conti correnti aperti. Il riferimento a tali oggetti è il
       parametro instance dei vari metodi. Gli accessi in lettura e scrittura
       al descrittore (accessi che vengono effettuati effettuati nei metodi
       della classe CC) sono esprimibili in "pure python" nel modo seguente,
       in cui b è il balance e X e il CC (CC=type(X)):
           X.b (lettura)         --> type(X).__dict__['b'].__get(X,type(X))
           X.b = 100 (scrittura) --> type(X).__dict__['b'].__set(X,100)
    '''
    def __init__(self):
        self.default = 0
        self.balances = {}
        
    def __get__(self,instance,cls):
        '''Usando come chiave l'istanza (di CC) passata come secondo 
           parametro, il metodo recupera il valore del saldo associato
           dal dizionario self.balances
        '''
        return self.balances.get(instance,self.default)
    
    def __set__(self, instance, value):
        '''Usando come chiave l'istanza (di CC) passata come secondo 
           parametro, il metodo "setta" il saldo corrispondente nel
           dizionario self.balances al valore value
        '''
        if value<-instance.credit:
            raise ValueError("Superato il valore del fido")
        self.balances[instance] = value
        
    def __delete__(self,instance):
        ''' 
        Cancella la coppia istanza/valore dal dizionario self.balances
        '''
        del self.balances[instance]

In [ ]:
class CC:
    """
    Classe per la gestione di un c/c:: v3.0
    Utilizza un descrittore per la memorizzazione e la manipolazione dei saldi.
    Introduce l'utilizzo del fido
    """
    def __init__(self,initBalance=0,BankCredit=0):
        ''' 
        Apre un conto con assegnato saldo iniziale e dato fido bancario
        '''
        self.credit = BankCredit
        self.b = initBalance
        
    def deposit(self,value):
        ''' 
        Deposita un valore positivo sul conto
        '''
        if value>0:
            self.b += value
            
    def withdraw(self, value):
        '''
        Ritira una data soma dal conto (i controlli sull'eventuale
        superamento del fido sono effettuati dal descrittore)
        '''
        self.b -= value
        
    def getbalance(self):
        '''
        Stampa il saldo
        '''
        print(f"Current balance is {self.b}")
    
    b = balance() # Il saldo è un descrittore

In [ ]:
X = CC()

In [ ]:
X.b

In [ ]:
# X.b = ...
type(X).__dict__['b'].__get__(X,type(X))

In [ ]:
# Versione getter esplicito
X.getbalance()

In [ ]:
X.deposit(100)
X.getbalance()

In [ ]:
X.withdraw(150)

In [ ]:
X.withdraw(50)
X.getbalance()

In [ ]:
Y = CC(100,500)
Y.withdraw(300)
Y.getbalance()
X.getbalance()